<a href="https://colab.research.google.com/github/RyanZha0/CIENE-4012/blob/main/Citibike%20NYC%20visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
file_path = '/content/drive/MyDrive/CIEN 4012 Project/Data/Citibike raw data/201906.csv'
data = pd.read_csv(file_path)

data

NameError: name 'pd' is not defined

In [ ]:
# Extract unique station locations from start and end stations
start_stations = data[['start station latitude', 'start station longitude']].drop_duplicates()
end_stations = data[['end station latitude', 'end station longitude']].drop_duplicates()

# Rename columns for merging
start_stations.columns = ['latitude', 'longitude']
end_stations.columns = ['latitude', 'longitude']

# Concatenate and drop duplicates to get unique stations
all_stations = pd.concat([start_stations, end_stations]).drop_duplicates()

# Check the result
all_stations.head(), len(all_stations)

(    latitude  longitude
 0  40.705312 -73.971001
 1  40.739448 -74.005070
 2  40.689407 -73.968855
 3  40.693261 -73.968896
 4  40.722281 -73.976687,
 788)

In [ ]:
import folium
from folium.plugins import HeatMap

# Create a base map
m = folium.Map(location=[40.73, -73.93], zoom_start=8

# Add a heatmap to the base map
HeatMap(all_stations).add_to(m)

# Display the map
m

In [ ]:
# Create a new base map
m_points = folium.Map(location=[40.73, -73.93], zoom_start=10)

# Add a marker for each station
for index, row in all_stations.iterrows():
    folium.CircleMarker(location=(row['latitude'], row['longitude']),
                        radius=3, # small radius for a simple dot
                        color='blue',
                        fill=True,
                        fill_color='blue').add_to(m_points)

# Display the map with points
m_points


In [ ]:
file_path1 = '/content/drive/MyDrive/CIEN 4012 Project/Codes/mobility data - during covid/Modified_Zip_Code_Tabulation_Areas__MODZCTA_.csv'
data = pd.read_csv(file_path1)
data.head(5)

,MODZCTA,label,ZCTA,pop_est,the_geom
0,10001,"10001, 10118","10001, 10119, 10199",23072,MULTIPOLYGON (((-73.98774438827894 40.74406551...
1,10002,10002,10002,74993,MULTIPOLYGON (((-73.99750380833868 40.71407144...
2,10003,10003,10003,54682,MULTIPOLYGON (((-73.98863576728334 40.72293099...
3,10026,10026,10026,39363,MULTIPOLYGON (((-73.96200720826798 40.80550908...
4,10004,10004,10004,3028,MULTIPOLYGON (((-74.00826694529984 40.70771979...


In [ ]:
!pip install shapely

In [ ]:
import pandas as pd
from shapely.geometry import Point
from shapely import wkt
import numpy as np

# Load the first dataset with station locations
file_path = '/content/drive/MyDrive/CIEN 4012 Project/Data/Citibike raw data/201906.csv'
data = pd.read_csv(file_path)

# Extract unique station locations from start and end stations
start_stations = data[['start station latitude', 'start station longitude']].drop_duplicates()
end_stations = data[['end station latitude', 'end station longitude']].drop_duplicates()

# Rename columns for merging
start_stations.columns = ['latitude', 'longitude']
end_stations.columns = ['latitude', 'longitude']

# Concatenate and drop duplicates to get unique stations
all_stations = pd.concat([start_stations, end_stations]).drop_duplicates()

# Load the second dataset with ZIP code polygons
zip_code_file_path = '/content/drive/MyDrive/CIEN 4012 Project/Codes/mobility data - during covid/Modified_Zip_Code_Tabulation_Areas__MODZCTA_.csv'
zip_codes_data = pd.read_csv(zip_code_file_path)

# Convert the_geom column to Shapely Polygon objects in zip_codes_data
zip_codes_data['polygon'] = zip_codes_data['the_geom'].apply(lambda wkt_str: wkt.loads(wkt_str))

# Initialize a column for zip codes in the all_stations data
all_stations['zipcode'] = np.nan

# Match zip codes to the station points
for idx, row in all_stations.iterrows():
    point = Point(row['longitude'], row['latitude'])
    for _, zip_row in zip_codes_data.iterrows():
        polygon = zip_row['polygon']
        if polygon.contains(point):
            all_stations.at[idx, 'zipcode'] = zip_row['MODZCTA']
            break

# Remove NaN values and count the number of stations per zipcode
stations_per_zipcode = all_stations.dropna(subset=['zipcode']).groupby('zipcode').size()

stations_per_zipcode.to_csv('stations.csv')


In [ ]:
import folium

# Create a map centered around an average location in the data
avg_lat = (data['start station latitude'].mean() + data['end station latitude'].mean()) / 2
avg_lon = (data['start station longitude'].mean() + data['end station longitude'].mean()) / 2
map_trip = folium.Map(location=[avg_lat, avg_lon], zoom_start=13)

# Drawing lines for each trip from start to end station
for _, row in data.iterrows():
    start_coords = [row['start station latitude'], row['start station longitude']]
    end_coords = [row['end station latitude'], row['end station longitude']]
    line = folium.PolyLine(locations=[start_coords, end_coords], weight=1, color='blue')
    map_trip.add_child(line)


map_trip
